### This genetic algorithm is an adaptation of [MourvanZhou's evolutionary algorithm code](https://github.com/MorvanZhou/Evolutionary-Algorithm/blob/master/tutorial-contents/Genetic%20Algorithm/Genetic%20Algorithm%20Basic.py) ###

In [34]:
from qutip import *
from scipy import arcsin, sqrt, pi
import numpy as np
import scipy.sparse as sp
from qutip.qobj import Qobj
%matplotlib
import matplotlib.pyplot as plt
import itertools
import copy
from math import log2
from scipy.optimize import minimize

Using matplotlib backend: Qt5Agg


In [35]:
INPUT_STATES = 16  # at least 16

In [38]:
def h(x):  #binary entropy
    return -x*log2(x) - (1-x)*log2(1-x)

def calc_XN(p):
    def calc_XN_tmp(q):
        tmp = h(q * (1-p)) - h(0.5*(1+sqrt(1-4*p*(1-p)*q**2)))
        return 1/tmp

    res = minimize(calc_XN_tmp, np.array([0.5]), method='nelder-mead', 
                       options={'xtol': 1e-8, 'disp': False});
    return 1/res.fun;


def rand_herm_mod(N, X, Y, density=1, dims=None):      #this is a modified version of the rand_herm function from QuTip
    if dims:
        _check_dims(dims, N, N)
    # to get appropriate density of output
    # Hermitian operator must convert via:
    herm_density = 2.0 * arcsin(density) / pi

    X_int = sp.rand(N, N, herm_density, format='csr')
    X_int.data = X - 0.5
    Y_int = X_int.copy()
    Y_int.data = 1.0j * Y - (0.5 + 0.5j)
    X_int = X_int + Y_int
    X_int.sort_indices()
    X_int = Qobj(X_int)
    if dims:
        return Qobj((X_int + X_int.dag()) / 2.0, dims=dims, shape=[N, N])
    else:
        return Qobj((X_int + X_int.dag()) / 2.0)
    

def rand_unitary_mod(N, X, Y, density=1, dims=None):    #this is a modified version of the rand_unitary function from QuTip
    #if dims:
    #    _check_dims(dims, N, N)
    U = (-1.0j * rand_herm_mod(N, X, Y, density)).expm()
    U.data.sort_indices()
    if dims:
        return Qobj(U, dims=dims, shape=[N, N])
    else:
        return Qobj(U)
    


def rand_ket(N, X, Y, density=1, dims=None):

#    if dims:
#        _check_dims(dims, N, 1)
    Xtmp = sp.rand(N, 1, density, format='csr')
    Xtmp.data = X - 0.5
    Ytmp = Xtmp.copy()
    Ytmp.data = 1.0j * Y - (0.5 + 0.5j)
    Xtmp = Xtmp + Ytmp
    Xtmp.sort_indices()
    Xtmp = Qobj(Xtmp)
    if dims:
        return Qobj(Xtmp / Xtmp.norm(), dims=dims, shape=[N, 1])
    else:
        return Qobj(Xtmp / Xtmp.norm())

    
    

def _check_dims(dims, N1, N2):   #this function is taken directly from QuTip
    if len(dims) != 2:
        raise Exception("Qobj dimensions must be list of length 2.")
    if (not isinstance(dims[0], list)) or (not isinstance(dims[1], list)):
        raise TypeError(
            "Qobj dimension components must be lists. i.e. dims=[[N],[N]]")
    if np.prod(dims[0]) != N1 or np.prod(dims[1]) != N2:
        raise ValueError("Qobj dimensions must match matrix shape.")
    if len(dims[0]) != len(dims[1]):
        raise TypeError("Qobj dimension components must have same length.")
        

def A_p2(rho): # this is the amplitude dampening channel on 2 qubits (2 uses). rho is a 2-qubit state.
    #------Moved to main to optimize code
    #KA = tensor(K1,K1)
    #KA_dag = tensor(K1,K1).dag()
    #KB = tensor(K1,K2)
    #KB_dag = tensor(K1,K2).dag()
    #KC = tensor(K2,K1)
    #KC_dag = tensor(K2,K1).dag()
    #KD = tensor(K2,K2)
    #KD_dag = tensor(K2,K2).dag()
    #--------

    return  KA * rho * KA_dag + KB * rho * KB_dag + KC * rho * KC_dag + KD * rho * KD_dag 


def Func(kets,ps):   #This is the estimate of X(N \tensor N)
    
    #rho_init = tensor(basis(2,0), basis(2,0)) * tensor(basis(2,0), basis(2,0)).dag()   #moved to main
    
    XNN = np.array([ entropy_vn(sum(px * A_p2(ket2dm(ketx)) for px,ketx in zip(ps[i],kets[i]))) 
                    - sum(px * entropy_vn(A_p2(ket2dm(ketx))) for px,ketx in zip(ps[i],kets[i])) for i in range(POP_SIZE)])

    return XNN
    
# find non-zero fitness for selection.
def get_fitness(pred): return pred + 1e-3 - np.min(pred)


def translateDNA(pop):   #pop is a list of 2 np arrays. One nparray for the population of X and Y, and one nparray for the population of ps
    XY = pop[0].dot(2 ** np.arange(DNA_SIZE)[::-1]) / (2**DNA_SIZE-1)
    #Us = np.array([[rand_unitary_mod(4, XY[i,j,0], XY[i,j,1], density=1, dims=[[2,2], [2,2]])  for j in range(INPUT_STATES)]  for i in range(POP_SIZE)]) 
    kets = np.array([[rand_ket(4, XY[i,j,0], XY[i,j,1], dims=[[2,2], [1,1]] )  for j in range(INPUT_STATES)]  for i in range(POP_SIZE)])
    prob = pop[1].dot(2 ** np.arange(DNA_SIZE)[::-1]) / (2**DNA_SIZE-1)
    ps = np.array([prob[i] / sum(prob[i]) for i in range(POP_SIZE)] ) 
    return [kets,ps]  #Us is an array of POP_SIZE arrays of 16 unitaries each

# nature selection wrt pop's fitness.
def select(pop, fitness):    
    idx = np.random.choice(np.arange(POP_SIZE), size=POP_SIZE, replace=True, p=fitness/fitness.sum())
    return [pop[0][idx], pop[1][idx]]

# mating process (genes crossover).
def crossover_and_mutate(individual_XY, individual_p, pop, mutate=False):
    #crossover
    if np.random.rand() < CROSS_RATE:
        i_ = np.random.randint(0, POP_SIZE, size=1)[0]                        # select another individual from pop
        cross_points_XY = np.random.randint(0, 2, size=INPUT_STATES*2*4*DNA_SIZE).astype(bool).reshape((INPUT_STATES,2,4,DNA_SIZE))# choose crossover points
        cross_points_p = np.random.randint(0, 2, size=DNA_SIZE*INPUT_STATES).astype(bool).reshape((INPUT_STATES,DNA_SIZE))# choose crossover points
        individual_XY[cross_points_XY] = pop[0][i_][cross_points_XY]
        individual_p[cross_points_p] = pop[1][i_][cross_points_p]
        
    #mutate
    if mutate:
        x_XY = np.random.choice([0, 1], size=INPUT_STATES*2*4*DNA_SIZE, 
                             p=[1-MUTATION_RATE, MUTATION_RATE]).astype(bool).reshape((INPUT_STATES,2,4,DNA_SIZE))
                                                      
        x_p = np.random.choice([0, 1], size=INPUT_STATES*DNA_SIZE, 
                             p=[1-MUTATION_RATE, MUTATION_RATE]).astype(bool).reshape((INPUT_STATES,DNA_SIZE))
                                                      

        individual_XY[x_XY] = np.abs(individual_XY[x_XY] - 1)  #flip the bits
        individual_p[x_p] = np.abs(individual_p[x_p] - 1)  #flip the bits
        
    return [individual_XY, individual_p]


In [ ]:
p = 0.3                 #probability p in the amp. damp. channel
DNA_SIZE = 15          # DNA length   # size of each number in the lists X and Y
POP_SIZE = 20           # population size
CROSS_RATE = 0.005        # mating probability (DNA crossover)
MUTATION_RATE = 0.001    # mutation probability
N_GENERATIONS = 5000

#---------------Moved from functions for optimization
K1 = basis(2,0) * basis(2,0).dag() + sqrt(1-p) * basis(2,1) * basis(2,1).dag()
K2 = sqrt(p) * basis(2,0) * basis(2,1).dag()
KA = tensor(K1,K1)
KA_dag = tensor(K1,K1).dag()
KB = tensor(K1,K2)
KB_dag = tensor(K1,K2).dag()
KC = tensor(K2,K1)
KC_dag = tensor(K2,K1).dag()
KD = tensor(K2,K2)
KD_dag = tensor(K2,K2).dag()

#rho_init = tensor(basis(2,0), basis(2,0)) * tensor(basis(2,0), basis(2,0)).dag()
#-------------------------

pop_XY = []
pop_p = []
for i in range(POP_SIZE):
    pop_XY.append(np.random.randint(2, size=([INPUT_STATES, 2, 4, DNA_SIZE])))
    pop_p.append(np.random.randint(2, size=([INPUT_STATES, DNA_SIZE])))
pop = [np.array(pop_XY), np.array(pop_p)]

#-----------------For plotting
plt.ion()
fig, ax = plt.subplots()
gen, holevo = [],[]
ax.scatter(gen,holevo)
plt.xlim(0,N_GENERATIONS)
plt.ylim(0.4,1.5)
XN = calc_XN(p) # this is X(N). You need to plot 2X(N)
plt.plot([i for i in range(N_GENERATIONS)], 
         [2*XN for j in range(N_GENERATIONS)])
plt.draw()
plt.xlabel("Generations")
plt.ylabel("X(N tensor N) estimate")
#-----------------------------

for _ in range(N_GENERATIONS):
    translated = translateDNA(pop)
    F_values = Func(translated[0], translated[1])    # compute function value by extracting DNA
    holevo.append(np.max(F_values))
    #-------plot
    ax.scatter(_, holevo[-1], c='red')
    plt.pause(0.05)
    #-------

    # GA part (evolution)
    fitness = get_fitness(F_values) #FIXED
    pop = select(pop, fitness)
    pop_copy = pop.copy()
    for parent_XY,parent_p in zip(pop[0], pop[1]):
        child_XY,child_p = crossover_and_mutate(parent_XY, parent_p, pop_copy, len(holevo)>1 and holevo[-1]==holevo[-2]) 
        parent_XY[:] = child_XY       # parent is replaced by its child
        parent_p[:] = child_p        # parent is replaced by its child

plt.ioff()